<a href="https://colab.research.google.com/gist/volmar86/4a190b7feaa475b5f741fb9f8d475753/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar
from ipywidgets import widgets, Layout, Button, Box, VBox, HBox
from IPython.display import display, clear_output, HTML

In [2]:
# https://discourse.jupyter.org/t/an-interactive-binder-config-file-builder-gui/1510
# Initialize your wage data here as a dictionary or a DataFrame.
wages_data = {
    '2019': {
        'EG 1': 2398, 'EG 2': 2463, 'EG 3': 2592.5, 'EG 4': 2722,
        'EG 5': 2884, 'EG 6': 3046, 'EG 7': 3240.5, 'EG 8': 3467.5,
        'EG 9': 3694.5, 'EG 10': 3937.5, 'EG 11': 4196.5, 'EG 12': 4488,
        'EG 13': 4780, 'EG 14': 5071.5, 'EG 15': 5363, 'EG 16': 5719.5,
        'EG 17': 6043.5
    },
    '2023': {
        'EG 1': 2522.5, 'EG 2': 2591, 'EG 3': 2727.5, 'EG 4': 2863.5,
        'EG 5': 3034, 'EG 6': 3204.5, 'EG 7': 3409, 'EG 8': 3648,
        'EG 9': 3886.5, 'EG 10': 4142.5, 'EG 11': 4414.5, 'EG 12': 4721.5,
        'EG 13': 5028.5, 'EG 14': 5335, 'EG 15': 5642, 'EG 16': 6017,
        'EG 17': 6358
    },
    '2024': {
        'EG 1': 2605.5, 'EG 2': 2676.5, 'EG 3': 2817.5, 'EG 4': 2958,
        'EG 5': 3134, 'EG 6': 3310, 'EG 7': 3521.5, 'EG 8': 3768.5,
        'EG 9': 4015, 'EG 10': 4279, 'EG 11': 4560, 'EG 12': 4877.5,
        'EG 13': 5194.5, 'EG 14': 5511, 'EG 15': 5828, 'EG 16': 6215.5,
        'EG 17': 6568
    },
    '2025': {
        'EG 1': 2709.5, 'EG 2': 2783.5, 'EG 3': 2930, 'EG 4': 3076.5,
        'EG 5': 3259.5, 'EG 6': 3442.5, 'EG 7': 3662.5, 'EG 8': 3919,
        'EG 9': 4175.5, 'EG 10': 4450, 'EG 11': 4742.5, 'EG 12': 5072.5,
        'EG 13': 5402.5, 'EG 14': 5731.5, 'EG 15': 6061, 'EG 16': 6464,
        'EG 17': 6830.5
    }
}

# Define the update dates for the wages
update_dates = {
    '2019': {'month': 8, 'day': 1},
    '2023': {'month': 6, 'day': 1},
    '2024': {'month': 5, 'day': 1},
    '2025': {'month': 5, 'day': 1}
    # ... Add other years ...
}

# Define the percentages for special payments
special_payments = {
    'Trafo': 0.184,
    'Urlaubsgeld': 15/21.75,
    'T-Zug A': 0.275,
    'T-Zug B': 0.185, # Assuming this is based on the EG7 wage
    # Weihnachtsgeld increases per year of employment, with a max of 100%
    'Weihnachtsgeld': {1: 0.3, 2: 0.4, 3: 0.5, 4: 0.6, 5: 0.65, 6: 0.7, 7: 0.75, 8: 0.8, 9: 0.85, 10: 0.9, 11: 0.95, 12: 1.0}
}


In [6]:
# Creating month options for the dropdown
months = {calendar.month_name[i]: i for i in range(1, 13)}

# Define widgets for the user input and Button to calculate in the end
widgets_dict = {
    'EG Level': widgets.Dropdown(options=[f'EG {i}' for i in range(1, 18)], value='EG 12', description='EG Level:', style={'description_width': 'initial'}, layout=Layout(width='150px')),
    'Tax Year': widgets.IntSlider(value=2024, min=2019, max=2025, description='Tax year:', style={'description_width': 'initial'}),
    'Performance Bonus': widgets.FloatSlider(value=9.0, min=0.0, max=15.0, step=1, description='Leistungszulage (%):', style={'description_width': 'initial'}),
    'Voluntary Bonus': widgets.FloatSlider(value=0.0, min=0.0, max=15.0, step=1, description='Freiwillige Zulage (%):', style={'description_width': 'initial'}),
    'Voluntary Bonus Start Month': widgets.Dropdown(options=months, value=1, description='Start Month:', style={'description_width': 'initial'}),
    'Liebherr Increase': widgets.FloatSlider(value=2.0, min=0.0, max=5.0, step=0.5, description='Liebherr Erhöhung (%):', style={'description_width': 'initial'}),
    'Liebherr Start Month': widgets.Dropdown(options=months, value=9, description='Start Month:', style={'description_width': 'initial'}),
    'Working Hours': widgets.FloatSlider(value=35, min=20, max=40, step=1, description='Working Hours:', style={'description_width': 'initial'}),
    'Working Hours Start Month': widgets.Dropdown(options=months, value=1, description='Start Month:', style={'description_width': 'initial'}),
    'Start Year': widgets.IntSlider(value=2023, min=2019, max=2025, description='Start of employment year:', style={'description_width': 'initial'}),
    #'Christmas Bonus Percentage': widgets.IntSlider(value=0, min=0, max=100, step=5, description='Percentage of Christmas Bonus:', style={'description_width': 'initial'}),
    'Calculate Wages': Button(description='Calculate Wages')
}

# Create a button to trigger the calculation
calculate_button = Button(description='Calculate Wages')

# Create the bar chart and summary table functions
def create_bar_chart(wages, regular_wages):
    # Clear the existing output explicitly every time this function is called
    chart_output.clear_output(wait=True)
    with chart_output:
        plt.figure(figsize=(10, 5)) # (12, 5) for legends outside of graph
        ax = plt.gca()  # Get current axes instance

        # Set axis colors
        ax.spines['bottom'].set_color('black')
        ax.spines['top'].set_color('black')
        ax.spines['right'].set_color('black')
        ax.spines['left'].set_color('black')

        # Set tick colors
        ax.tick_params(axis='x', colors='black')
        ax.tick_params(axis='y', colors='black')

        # Set label colors
        ax.xaxis.label.set_color('black')
        ax.yaxis.label.set_color('black')

        # Filter out 'Yearly Total' from the wages dictionary
        monthly_wages = {month: wage for month, wage in wages.items() if month != 'Yearly Total'}
        # Prepare to track the bars for the legend
        bars_regular = None
        bars_extra = None
        # Calculate the portion of each month's wage up to and exceeding the January wage
        for index, (month, wage) in enumerate(monthly_wages.items(), start=1):
            if wage > regular_wages[index-1]:
                bars_regular = plt.bar(month, regular_wages[index-1], color='green')
                bars_extra = plt.bar(month, wage - regular_wages[index-1], color='cyan', bottom=regular_wages[index-1])
            else:
                bars_regular = plt.bar(month, wage, color='green')

        plt.xlabel('Month')
        plt.ylabel('Wage (€)')
        plt.title('Monthly Wage Distribution')
        plt.xticks(rotation=45)
        plt.yticks(np.arange(0, max(monthly_wages.values()) + 1000, 2000))  # Set y-ticks with an interval of 1,000
        plt.grid(axis='y')  # Add a grid on the y-axis
        # Adding legend
        # loc='upper right', bbox_to_anchor=(1.1, 1.0)) for legends outside of graph
        plt.legend([bars_regular, bars_extra], ['Regular Monthly Wage', 'Extra Payments'], loc='upper left')
        plt.tight_layout()
        plt.draw()  # Force redraw in case the chart gets not updated
        plt.show()

def create_summary_table(wages):
    # Clear the existing output explicitly every time this function is called
    table_output.clear_output(wait=True)
    with table_output:
        # Creating a DataFrame from the wages dictionary
        df = pd.DataFrame(list(wages.items()), columns=['Month', 'Total Wage (€)'])

        # If 'Yearly Total' is in the dictionary, move it to the end
        if 'Yearly Total' in df['Month'].values:
            yearly_total_row = df[df['Month'] == 'Yearly Total']
            df = df[df['Month'] != 'Yearly Total']
            df = pd.concat([df, yearly_total_row], ignore_index=True)

        # Display the DataFrame without the index
        #print(df.to_string(index=False))
        display(HTML(df.to_html(index=False)))

# The actual function to calculate the monthly salary
def calculate_monthly_wages(param_dict):
    # Extract parameters from the dict
    eg_level = param_dict['EG Level']
    tax_year = param_dict['Tax Year']
    leistungszulage_percent = param_dict['Performance Bonus']
    freiwillige_zulage = param_dict['Voluntary Bonus']
    f_z_month = param_dict['Voluntary Bonus Start Month']
    liebherr_erhoehung = param_dict['Liebherr Increase']
    cmp_month = param_dict['Liebherr Start Month']
    working_hours = param_dict['Working Hours']
    wh_month = param_dict['Working Hours Start Month']
    employment_year = param_dict['Start Year']
    wages = {}
    total_annual_wage = 0
    # Prepare the list for monthly_wages (wages that one receives no matter
    # what in a month)
    regular_wages = []

    # Function to find the wage for a given Entgeltgruppe, year, and month
    def find_wage_for_month(eg_level, year, month):
        applicable_year = year
        while str(applicable_year) not in wages_data:
            applicable_year -= 1

        update_year = str(applicable_year)
        if month < update_dates[update_year]['month']:
            applicable_year -= 1
            while str(applicable_year) not in wages_data:
                applicable_year -= 1

        return wages_data[str(applicable_year)][eg_level]

    # Iterate through each month
    for month in range(1, 13):
        base_wage = find_wage_for_month(eg_level, tax_year, month)
        # calculate the leistungszulage in january, update it only after the
        # company raise. Else, update it, and it will stay so until the end
        if month ==1:
          leistungszulage = base_wage * leistungszulage_percent / 100
        elif month == cmp_month:
            leistungszulage_percent += liebherr_erhoehung
            leistungszulage = base_wage * leistungszulage_percent / 100
        monthly_wage = base_wage + leistungszulage

        # Apply freiwillige zulage
        if month >= f_z_month:
          monthly_wage *= 1 + freiwillige_zulage / 100

        # Apply special payments
        if month == 2:  # February (Trafo)
            monthly_wage *= 1 + special_payments['Trafo']
        elif month == 6:  # June (Urlaubsgeld)
            monthly_wage *= 1 + special_payments['Urlaubsgeld']
        elif month == 7:  # July (T-Zug)
            t_zug_b_amount = find_wage_for_month("EG 7", tax_year, 7) * special_payments['T-Zug B']
            monthly_wage = monthly_wage * (1 + special_payments['T-Zug A']) + t_zug_b_amount
        elif month == 11:  # November (Weihnachtsgeld)
            years_of_employment = tax_year - employment_year + 1
            weihnachtsgeld_percent = special_payments['Weihnachtsgeld'].get(years_of_employment, 1.0)
            monthly_wage *= 1 + weihnachtsgeld_percent

        # Apply working hours adjustment
        if month >= wh_month:
          monthly_wage *= working_hours / 35
        else:
          monthly_wage *= working_hours / 35

        # Store the calculated wage
        monthly_wage = round(monthly_wage, 2)
        wages[calendar.month_name[month]] = monthly_wage
        total_annual_wage += monthly_wage
        # Add the total annual wage to the wages dictionary
        wages['Yearly Total'] = round(total_annual_wage, 2)
        # add the normal monthly wage to the list
        m_w = (base_wage + leistungszulage)*(1 + freiwillige_zulage / 100)*\
                (working_hours / 35)
        regular_wages.append(m_w)

    return wages, regular_wages

# Button click handler (callback function for the button)
def on_calculate_clicked(b):
    # Gather parameters from widgets and calculate wages
    params = {key: widget.value for key, widget in widgets_dict.items() if key != 'Calculate Wages'}
    wages, r_wages = calculate_monthly_wages(params)
    create_bar_chart(wages, r_wages)
    create_summary_table(wages)

# Binding the button to the function
widgets_dict['Calculate Wages'].on_click(on_calculate_clicked)

# Organizing widgets in the layout
# Define the style for centering content in a VBox or HBox
center_layout = widgets.Layout(display='flex', justify_content='center', align_items='center')
# Selection Box
selection_box = VBox([
    widgets_dict['EG Level'],
    widgets_dict['Tax Year'],
    widgets_dict['Performance Bonus'],
    HBox([widgets_dict['Voluntary Bonus'], widgets_dict['Voluntary Bonus Start Month']]),
    HBox([widgets_dict['Liebherr Increase'], widgets_dict['Liebherr Start Month']]),
    HBox([widgets_dict['Working Hours'], widgets_dict['Working Hours Start Month']]),
    widgets_dict['Start Year'],
    widgets_dict['Calculate Wages']
])
# Placeholders for the chart and the table
chart_output = widgets.Output()
table_output = widgets.Output()

# Create a VBox for the left box with a specific layout to center its contents
selection_centered = VBox([selection_box], layout=center_layout)

# Use the center_layout for the table_output VBox as well
table_output_centered = VBox([table_output], layout=center_layout)

# Main app layout
# Modify the main app layout to apply centering
app_layout = VBox([
    selection_centered,
    chart_output,  # chart_output remains as is, you can wrap it with VBox if centering is needed
    table_output_centered
], layout=center_layout)

# Display the entire app layout
display(app_layout)


In [4]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package,
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

ipywidgets==7.7.1
matplotlib==3.7.1
numpy==1.25.2
pandas==2.0.3
